In [2]:
import numpy as np 
import pandas as pd
from gensim.models.fasttext import FastText
from gensim.models.fasttext import load_facebook_model
import random
import os
import re
import spacy
! pip install reportlab
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet', "/kaggle/working/nltk_data/")
nltk.download('omw-1.4', "/kaggle/working/nltk_data/")
! unzip /kaggle/working/nltk_data/corpora/wordnet.zip -d /kaggle/working/nltk_data/corpora
! unzip /kaggle/working/nltk_data/corpora/omw-1.4.zip -d /kaggle/working/nltk_data/corpora
nltk.data.path.append("/kaggle/working/nltk_data/")



In [3]:
# load pretraind model
from gensim.models.fasttext import load_facebook_model
! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
! gunzip "cc.en.300.bin.gz"
# Load the pretrained FastText model
pretrained_fastText_en = load_facebook_model('/kaggle/working/cc.en.300.bin')

--2024-04-20 03:17:36--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.162.163.51, 3.162.163.11, 3.162.163.34, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.162.163.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: 'cc.en.300.bin.gz'

cc.en.300.bin.gz    100%[===================>]   4.19G   186MB/s    in 22s     

2024-04-20 03:17:58 (199 MB/s) - 'cc.en.300.bin.gz' saved [4503593528/4503593528]



In [4]:
yelpData = pd.read_json("/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json", lines=True)

yelpData

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0
...,...,...,...,...,...
908910,eYodOTF8pkqKPzHkcxZs-Q,3lHTewuKFt5IImbXJoFeDQ,Disappointed in one of your managers.,2021-09-11 19:18:57,0
908911,1uxtQAuJ2T5Xwa_wp7kUnA,OaGf0Dp56ARhQwIDT90w_g,Great food and service.,2021-10-30 11:54:36,0
908912,v48Spe6WEpqehsF2xQADpg,hYnMeAO77RGyTtIzUSKYzQ,Love their Cubans!!,2021-11-05 13:18:56,0
908913,ckqKGM2hl7I9Chp5IpAhkw,s2eyoTuJrcP7I_XyjdhUHQ,Great pizza great price,2021-11-20 16:11:44,0


In [20]:

text = list(yelpData['text'])
text=text[0:2000]
print(format(text[0:30]))

['Avengers time with the ladies.', 'They have lots of good deserts and tasty cuban sandwiches', "It's open even when you think it isn't", 'Very decent fried chicken', 'Appetizers.. platter special for lunch', 'Chili Cup + Single Cheeseburger with onion, pickle, and relish + Vanilla Coca-Cola...so far.', "Saturday, Dec 7th 2013, ride Patco's Silver Sleigh w/ Santa & his elves on a decorated train into Center City. Trains leave from Lindenwold at 10am, 11:15am, & 12:30pm, and make all stops. Great for kids!", 'This is probably the best place in the cool Springs area to watch a game and eat', 'Tacos', 'Starbucks substitute in boring downtown Tampa. Ugh. Never again!', 'Order the Tortilla Soup', 'Very good will definitely be coming back!!', 'If the Hotlight is on you must stop in.', "Let's go Yankees!", "Basically same food as rally's for $5 more", "Don't go for dinner. They close at 6. Really Yvonne L? Negative stars for a restaurant where you you didn't and never intended to eat?", '30 m

In [21]:


def clean_text(text):
    lemmatizer = WordNetLemmatizer()

    words = [
      lemmatizer.lemmatize(word) for word in word_tokenize(text.lower())
      if len(word) > 3 and not word in stopwords.words('english')
      and word.isalpha()
  ]

    return words


In [22]:
text = [ clean_text(word) for word in text]
print((text[0:30]))

[['avenger', 'time', 'lady'], ['lot', 'good', 'desert', 'tasty', 'cuban', 'sandwich'], ['open', 'even', 'think'], ['decent', 'fried', 'chicken'], ['platter', 'special', 'lunch'], ['chili', 'single', 'cheeseburger', 'onion', 'pickle', 'relish', 'vanilla'], ['saturday', 'ride', 'patco', 'silver', 'sleigh', 'santa', 'elf', 'decorated', 'train', 'center', 'city', 'train', 'leave', 'lindenwold', 'make', 'stop', 'great', 'kid'], ['probably', 'best', 'place', 'cool', 'spring', 'area', 'watch', 'game'], ['taco'], ['starbucks', 'substitute', 'boring', 'downtown', 'tampa', 'never'], ['order', 'tortilla', 'soup'], ['good', 'definitely', 'coming', 'back'], ['hotlight', 'must', 'stop'], ['yankee'], ['basically', 'food', 'rally'], ['dinner', 'close', 'really', 'yvonne', 'negative', 'star', 'restaurant', 'never', 'intended'], ['min', 'take', 'right', 'right', 'tight'], ['grilled', 'chicken', 'entree', 'delicious', 'small', 'portion', 'like', 'environment', 'sure', 'though'], ['bomb', 'love', 'dish', 

In [24]:
# Train FastText model
fast_text_model = FastText(sentences=text,vector_size=100, window=5, min_count=1, workers=4, sg=1,epochs=500)

In [ ]:
import random
from gensim.models import FastText

# Get a list of all words in the model's vocabulary
all_words = list(fast_text_model.wv.index_to_key)
random_words = random.sample(all_words, 10)



In [31]:

# For each random word, find the top 10 similar and dissimilar words
for word in random_words:
    print(f"Word: {word}")
    print("Top 10 similar words:")
    similar_words = fast_text_model.wv.most_similar(word, topn=10)
    for similar_word, similarity in similar_words:
        print(f"  {similar_word}: {similarity:.2f}")
    sorted_words = sorted(all_words, key=lambda x: fast_text_model.wv.similarity(word, x))
    print("Top 10 dissimilar words:")
    for dissimilar_word in sorted_words[-10:]:
        dissimilarity = fast_text_model.wv.similarity(word, dissimilar_word)
        print(f"  {dissimilar_word}: {dissimilarity:.2f}")
    print("\n")

Word: benoit
Top 10 similar words:
  benefit: 0.82
  benton: 0.79
  beard: 0.70
  suit: 0.68
  knit: 0.67
  twang: 0.67
  trentaaaaa: 0.66
  beotch: 0.66
  brat: 0.65
  pocket: 0.65
Top 10 dissimilar words:
  brat: 0.65
  beotch: 0.66
  trentaaaaa: 0.66
  twang: 0.67
  knit: 0.67
  suit: 0.68
  beard: 0.70
  benton: 0.79
  benefit: 0.82
  benoit: 1.00


Word: cocktail
Top 10 similar words:
  mocktails: 0.96
  vieux: 0.75
  scratch: 0.75
  carre: 0.73
  tail: 0.73
  instinct: 0.73
  mahi: 0.72
  penny: 0.69
  definitely: 0.65
  winner: 0.65
Top 10 dissimilar words:
  definitely: 0.65
  penny: 0.69
  mahi: 0.72
  instinct: 0.73
  tail: 0.73
  carre: 0.73
  scratch: 0.75
  vieux: 0.75
  mocktails: 0.96
  cocktail: 1.00


Word: definite
Top 10 similar words:
  definitely: 0.92
  definately: 0.72
  returning: 0.70
  fireball: 0.69
  fiercefinch: 0.69
  riviera: 0.68
  note: 0.67
  parrrrttttyyyy: 0.66
  hogwarts: 0.66
  onsite: 0.66
Top 10 dissimilar words:
  hogwarts: 0.66
  parrrrttttyyyy

In [33]:
for word in random_words:
    print(f"Word: {word}")
    print("Top 10 similar words:")
    similar_words = pretrained_fastText_en.wv.most_similar(word, topn=10)
    for similar_word, similarity in similar_words:
        print(f"  {similar_word}: {similarity:.2f}")
        
    sorted_words = sorted(pretrained_fastText_en.wv.index_to_key, key=lambda x: pretrained_fastText_en.wv.similarity(word, x))
    print("Top 10 dissimilar words:")
    for dissimilar_word in sorted_words[-10:]:
        dissimilarity = pretrained_fastText_en.wv.similarity(word, dissimilar_word)
        print(f"  {dissimilar_word}: {dissimilarity:.2f}")
    print("\n")

Word: benoit
Top 10 similar words:
  gauthier: 0.66
  cormier: 0.63
  santino: 0.62
  Beniot: 0.62
  heyman: 0.62
  lawler: 0.61
  christophe: 0.61
  batista: 0.61
  Benoit: 0.61
  claude: 0.61
Top 10 dissimilar words:
  Benoit: 0.61
  batista: 0.61
  christophe: 0.61
  lawler: 0.61
  heyman: 0.62
  Beniot: 0.62
  santino: 0.62
  cormier: 0.63
  gauthier: 0.66
  benoit: 1.00


Word: cocktail
Top 10 similar words:
  cocktails: 0.85
  coctail: 0.73
  Cocktail: 0.71
  martini: 0.70
  martinis: 0.68
  cocktail.The: 0.67
  mojito: 0.66
  cocktailing: 0.66
  mocktail: 0.65
  cocktail.: 0.65
Top 10 dissimilar words:
  mocktail: 0.65
  cocktailing: 0.66
  mojito: 0.66
  cocktail.The: 0.67
  martinis: 0.68
  martini: 0.70
  Cocktail: 0.71
  coctail: 0.73
  cocktails: 0.85
  cocktail: 1.00


Word: definite
Top 10 similar words:
  definate: 0.72
  Definite: 0.68
  DEFINITE: 0.65
  definte: 0.64
  deﬁnite: 0.54
  definitive: 0.54
  indication: 0.54
  hint: 0.53
  slight: 0.53
  appreciable: 0.53
T